In [11]:
# first check the wd
# this should be AIggregation folder

import os
os.getcwd()

#os.chdir("..")


'c:\\Users\\obiew\\Desktop\\github\\AIggregation'

In [12]:


#now import fiftyone
import fiftyone as fo
#from fiftyone import ViewField as F



# Import annotated test image into fiftyone dataset type
dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.YOLOv5Dataset,
    yaml_path = "datasets/test_image/test.yaml"
)

# open test image instance to verify it loaded correctly
session = fo.launch_app(dataset)



 100% |█████████████████████| 1/1 [50.7ms elapsed, 0s remaining, 20.7 samples/s] 


07/11/2024 13:04:29 - INFO - eta.core.utils -    100% |█████████████████████| 1/1 [50.7ms elapsed, 0s remaining, 20.7 samples/s] 


In [13]:
# import required sahi functions
from sahi import AutoDetectionModel
from sahi.predict import get_prediction, get_sliced_prediction, predict

os.getcwd()


# specify AI detection model
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov5',
    model_path='AIggregation_yolov5m/weights/best.pt', #specify path to trained model
    confidence_threshold=0.25,
    device="cpu", # or 'cuda:0'
)


# do a simple yolo prediction without slicing
# only on the first image in the dataset
result = get_prediction(dataset.first().filepath, detection_model)


# now do sliced prediction with sahi
sliced_result = get_sliced_prediction(
    dataset.first().filepath,
    detection_model,
    slice_height = 608,
    slice_width = 608,
    overlap_height_ratio = 0.1,
    overlap_width_ratio = 0.1
)


# compare number of detections with and without slicing just for fun
num_sliced_dets = len(sliced_result.to_fiftyone_detections())
num_orig_dets = len(result.to_fiftyone_detections())

print(f"Detections predicted without slicing: {num_orig_dets}")
print(f"Detections predicted with slicing: {num_sliced_dets}")




TypeError: ('model_path is not a valid yolov5 model path: ', AttributeError("module 'yolov5' has no attribute 'load'"))

In [ ]:



# Now to run and compare different slicing parameters
def predict_with_slicing(sample, label_field, **kwargs):
    result = get_sliced_prediction(
        sample.filepath, detection_model, verbose=0, **kwargs
    )
    sample[label_field] = fo.Detections(detections=result.to_fiftyone_detections())



kwargs = {"overlap_height_ratio": 0.2, "overlap_width_ratio": 0.2}

for sample in dataset.iter_samples(progress=True, autosave=True):
    predict_with_slicing(sample, label_field="small_slices", slice_height=304, slice_width=304, **kwargs)
    predict_with_slicing(sample, label_field="large_slices", slice_height=608, slice_width=608, **kwargs)


#bring up new session to see boxes
session.show()



In [ ]:

# evaluate the different sahi predictions
large_slice_results = dataset.evaluate_detections("large_slices", gt_field="ground_truth", eval_key="eval_large_slices")
small_slice_results = dataset.evaluate_detections("small_slices", gt_field="ground_truth", eval_key="eval_small_slices")


# print eval results
print("-" * 50)
print("Large slice results:")
large_slice_results.print_report()

print("-" * 50)
print("Small slice results:")
small_slice_results.print_report()




In [1]:
import os
os.getcwd()


'c:\\Users\\obiew\\Desktop\\github\\AIggregation'